In [ ]:
import numpy as np
import cv2 as cv
import glob
from constants import *
import os
from time import sleep
world_footage_path = os.path.join(recordings_folder, "camera_calibrations","world_footage")
depth_footage_path = os.path.join(recordings_folder, "camera_calibrations","depth_footage")

## record depth camera

In [ ]:
"""
import pyrealsense2 as rs
#record frames into depth footage path as png's

depth_footage_path =depth_footage_path + "_2"
print(depth_footage_path)
rs_config = rs.config()
rs_config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)

pipeline = rs.pipeline()
pipeline.start(rs_config)


for i in range(20):
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    cv.imwrite(os.path.join(depth_footage_path, str(i) + ".png"), color_image)

#only keep 20% of the frames
depth_frames = glob.glob(os.path.join(depth_footage_path, "*.png"))
depth_frames.sort(key=os.path.getmtime)
for i in range(len(depth_frames)):
    if i % 5 != 0:
        os.remove(depth_frames[i])
"""

In [ ]:
"""
#extract frames from world footage path as png's
world_cam_path = os.path.join(world_footage_path,"world_vid.mp4")
video = cv.VideoCapture(world_cam_path)
fps = video. get(cv. CAP_PROP_FPS)
print(fps)
i = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if i %12 == 0:
        cv.imwrite(os.path.join(world_footage_path, str(i) + ".png"), frame)
    i += 1
"""


## world camera

In [ ]:
#calibrate the world camera with checkerboard images with cv2
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:11].T.reshape(-1,2)*60                        #60mm between each square
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(os.path.join(world_footage_path,'*.png'))
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,11), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,11), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
#print all the calibration parameters
print("ret: ", ret)
print("mtx: ", mtx)
print("dist: ", dist)
print("rvecs: ", rvecs)
print("tvecs: ", tvecs)

### calibrate distorsion

In [ ]:
img = cv.imread(os.path.join(world_footage_path, '108.png'))
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_world.png")
cv.imwrite(calibration_path, dst)

In [ ]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_world_2.png")
cv.imwrite(calibration_path, dst)

## Depth camera

In [ ]:
#calibrate the world camera with checkerboard images with cv2
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:11].T.reshape(-1,2)*60                        #60mm between each square
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(os.path.join(depth_footage_path,'*.png'))
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,11), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,11), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
#print all the calibration parameters
print("ret: ", ret)
print("mtx: ", mtx)
print("dist: ", dist)
print("rvecs: ", rvecs)
print("tvecs: ", tvecs)

In [ ]:
img = cv.imread(os.path.join(depth_footage_path, '100.png'))
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_depth.png")
cv.imwrite(calibration_path, dst)

In [ ]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
calibration_path = os.path.join(recordings_folder, "camera_calibrations","calibration_depth_2.png")
cv.imwrite(calibration_path, dst)